In [38]:
# Neural network to identify the flowers type using the Iris data 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn import linear_model, metrics, model_selection, datasets

iris= datasets.load_iris()
x_train, x_test, y_train, y_test = model_selection.train_test_split(iris.data,iris.target,test_size=0.2,random_state=0)

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def devsigmoid(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

class arrays(object):
    """To generate the ramdon matrices for each layer"""
    def __init__(self, row=None, col=None,seeds=None):
        self.row = row
        self.col = col
        self.seeds= seeds
          
    def matrix(self):
        np.random.seed(self.seeds)
        if self.col == 1:
            return np.random.rand(self.row)
        elif self.row == 1:
            return np.random.rand(self.col)
        else:
            return np.random.rand(self.row,self.col)
        
    def vec(self):
        np.random.seed(77*self.seeds+29)
        return np.random.rand(self.row)
    
def resta(v1,v2):
    """Function to substract vectors of any size.
        len(v1) = len(v2)"""    
    if len(v1) > 1:
        vec=[]
        for i in range(len(v1)):
            vec.append((v1[i]-v2[i]))
    else:
        vec= np.array(v1)-np.array(v2)
    return np.array(vec)

def transres(values,options):
    """Transform data input results into vectors of
       the size of the element of the last layer"""    
    if options == 1 :
        return np.array(values)
    else:
        newvec= np.zeros(options)
        newvec[values]= 1
        return newvec

#**************************************************************#
#---------------------- backpropagation -----------------------#
#------------------------------------------------ -------------#   
def backprop(M,bb,valout,y,yp,siz,eta):
    """ M = coeficcients matrix
         bb = bias 
         valout = value predicted
         y = data input (previous value got from feedforward)
         yp = value expected """
#--------------------------- Iniciando ------------------------#
#------------------- Coded to get the first delta -------------#
    dim0 = np.reshape(M[siz-2], (len(M[siz-2]), -1))
    xsiz,ysiz = np.shape(dim0)
    cost = resta(valout,yp)
    if ysiz <= 1:
        z = np.dot(M[siz-2].T,y[siz-2]) + bb[siz-2]
        delta=2*cost*devsigmoid(z)
    else:  
        z = np.matmul(M[siz-2],y[siz-2]) + bb[siz-2]
        #print(np.shape(M[siz-2]),np.shape(y[siz-2]),np.shape(bb[siz-2]),np.shape(z))
        z1=len(z)
        delta0 =[]
        for i in range(z1):
            delta0.append(2*cost[i]*devsigmoid(z[i]))
        delta = np.reshape(np.array(delta0),(z1,1)) 
#------------------------------------------------------------------#
    for k in range(siz-2,-1,-1):
        nabla_w1 = []
        y2 = y[k]
        siz1,siz2 = len(y2),len(delta)
        bb[k] = np.array(bb[k])
        for i in range(siz1):
            for j in range(siz2):
                nabla_w1.append(eta*y2[i]*delta[j]) 
        nabla_w = np.reshape(np.array(nabla_w1),(siz1,siz2))        
        nabla_b = (eta*delta)
        if len(np.atleast_1d(delta))==1:
            suma = M[k]*delta
            M[k] = (M[k] - nabla_w )
        else:
            suma = np.matmul(M[k].T,delta)
            M[k] = (M[k] - nabla_w.T )
        bb[k] = bb[k] - nabla_b
        if k >=1 :
            y2= np.reshape(y[k-1], (len(y[k-1]),1))
            z = np.matmul(M[k-1],y2) + bb[k-1]
            junto= []
            for j in range(len(z)):
                junto.append(suma[j]*devsigmoid(z[j]))
            delta= np.reshape(np.array(junto),(len(z),1))
            nabla_w = []       
    return (M, bb)   

#**************************************************************#
#----------------------- feedforward --------------------------#
#------------------------------------------------ -------------# 
def feedforw(w,vals,bbb,sizsys):
        fval=[]
        vals= np.reshape(vals,(len(vals),1))
        for j in range(sizsys-1):
            w[j] = np.array(w[j])
            if len(np.atleast_1d(bbb[j]))== 1:
                valor= []
                bbb[j]= bbb[j]
                w[j] = np.reshape(w[j], (len(w[j]), -1))
                vals = np.dot(w[j].T,vals) + np.array(bbb[j])
                for i in range(len(vals)):
                    valor.append(sigmoid(vals[i]))
                vals= np.array(valor) 
            else:
                valor= []
                bbb[j] = np.reshape(bbb[j],(len(bbb[j]),1))
                vals = np.matmul(w[j],vals) + np.array(bbb[j])
                for i in range(len(vals)):
                    valor.append(sigmoid(vals[i]))
                vals= np.array(valor)  
            fval.append(vals)   
        return (np.array(fval))

#------------------------------------------------------------------------#
#**************************  Neuronal Network   *************************# 
#------------------------------------------------------------------------#
#------------------------------------------------------------------------#
def network(vec,indat,res,eta,tol):
    """ vec is a vector that contains the number of neurons in each layer. 
        The input parameters are considered as the first layer """
    
    siz = len(vec)
    sizdata = len(indat)
    vals= np.array(indat) 
    wi, bi = [], [] 
    
#--------------------- initialization ---------------------#  
    for j in range(siz-1):
        mm= arrays(vec[j+1],vec[j],j+7)
        ww, bb = mm.matrix(), mm.vec()
        wi.append(ww)
        bi.append(bb)  
#-----------------------------------------------------------       
    epoca= 0 
    epoch = 1000    # Number of iterations of the method
    while True:
        eerr= True
        for kk in range(sizdata):
            yi= []
            yi.append(vals[kk,:])            
            vals1 = vals[kk]
            results = transres(res[kk],vec[siz-1])
            y2= (feedforw(wi,vals1,bi,siz))
            for i in range(len(y2)):
                yi.append(y2[i])
            respred = yi[siz-1]
            erro= resta(respred,results)
            erreur = (erro*erro).mean()
            if erreur >= tol:
                wi, bi = backprop(wi,bi,respred,yi,results,siz,eta)
                eerr=True
            wf, bf = wi, bi       
        epoca+=1 
        if (epoca%50==0 ):
            print('epoca: ',epoca,', error: ',erreur)
            
        if eerr==False or epoca== epoch:
            break                     
    return (wf,bf)
#-----------------------------------------------------------------------#
#------------------------------ END  -----------------------------------#

Nlayer1 = len(x_train[0]) # To know the size of the first layer
red = [Nlayer1,3,3,3]
eta, tol = 0.05, 1e-4
print("The results start here")
wajust, bajust = network(red,x_train,y_train,eta,tol)
#print(wajust,'W matrix','\n')
#print(bajust,'b matrix','\n')


def predict(x_teste, wajust,bajust):
    """ prediction """
    siz= np.shape(x_teste)
    layersnum = len(red)
    vals = np.array(x_teste)
    sol= []
    for k in range(siz[0]):
        vals1 = vals[k]  
        values = feedforw(wajust,vals1,bajust,layersnum)
        sol.append(values[layersnum-2])
    return np.array(sol)

iden = predict(x_test, wajust,bajust)
#iden = predict(x_test[0:3,:], wajust,bajust)
#print(x_test[0:3,:],y_test[0:3])
#print(iden)
#print('*************************************************************')
#------------ Comparison ----------------#
#for i in range(len(y_test)):
 #   print(iden[i],y_test[i])

def accuracia(vec,valpre, known):
    total = 0
    errequa = 0.005
    siz = len(vec)
    sizt= len(known)
    for i in range(sizt):
        known2 = transres(known[i],vec[siz-1])
        erro= resta(valpre[i],known2)
        erf = (erro*erro).mean()
        if erf <= errequa:
            total += 1
        else:
            pass
    return total / sizt  

print('accuracy of the method: ',accuracia(red,iden, y_test)) 
#print('accuracy of the method: ',accuracia(red,iden, y_test[0:3])) 


The results start here
epoca:  50 , error:  0.21874157400801075
epoca:  100 , error:  0.21915813223792854
epoca:  150 , error:  0.21964107496965235
epoca:  200 , error:  0.21998556012335255
epoca:  250 , error:  0.1206235457193977
epoca:  300 , error:  0.011341022504714469
epoca:  350 , error:  0.024747153049116438
epoca:  400 , error:  0.013012578418722644
epoca:  450 , error:  0.007680585283187957
epoca:  500 , error:  0.0063527866169911205
epoca:  550 , error:  0.003432620416302118
epoca:  600 , error:  0.004786877075184415
epoca:  650 , error:  0.0038437456149292906
epoca:  700 , error:  0.0028569641964209373
epoca:  750 , error:  0.002933990944857558
epoca:  800 , error:  0.003921965433652414
epoca:  850 , error:  0.0031351119403236517
epoca:  900 , error:  0.002345266771351138
epoca:  950 , error:  0.0023265289534448036
epoca:  1000 , error:  0.0023639884626219213
accuracy of the method:  0.8
